In [1]:
import pandas as pd
import os
import glob
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from IPython.display import HTML

In [2]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [3]:
path=r'C:\Users\stang\Google Drive\Data_for_fun\Fantasy Basketball\DailyGameLogs'
allFiles = glob.glob(path + "/*.csv")

gamelogs = pd.concat([pd.read_csv(f, encoding='cp1252') for f in glob.glob('*.csv')])

In [4]:
path1=r'C:\Users\stang\Google Drive\Data_for_fun\Fantasy Basketball'

gamelogs.to_csv(os.path.join(path1,r'GameLogs.csv'))

In [5]:
gamelogs['Date'] = pd.to_datetime(gamelogs['Date'])
gamelogs.sort_values(by=['Player','Date'], inplace=True, ascending=False) 
# Want latest game to be game 1 - easy filter for last 10 games
gamelogs['Game_Num'] = gamelogs.groupby('Player').cumcount() + 1

### Full Season

In [6]:
data_group = gamelogs.groupby(by=["Player"]).agg({'GmSc':'size','PTS':['sum'],'TRB':['sum'],'BLK':['sum'], 'STL':['sum'], \
                                              'AST':['sum'], '3P':['sum'], 'FG':['sum'], 'FGA':['sum'], 'FT':['sum'], \
                                              'FTA':['sum'], 'TOV':['sum']})

In [7]:
data_group["PTS_G"] = data_group["PTS", "sum"] / data_group["GmSc", "size"]
data_group["TRB_G"] = data_group["TRB", "sum"] / data_group["GmSc", "size"]
data_group["BLK_G"] = data_group["BLK", "sum"] / data_group["GmSc", "size"]
data_group["STL_G"] = data_group["STL", "sum"] / data_group["GmSc", "size"]
data_group["AST_G"] = data_group["AST", "sum"] / data_group["GmSc", "size"]
data_group["3P_G"] = data_group["3P", "sum"] / data_group["GmSc", "size"]

data_group["FGA"] = np.where(data_group["FGA"] == 0, 1, data_group["FGA"])
data_group["FG%"] = data_group["FG", "sum"]**2 / data_group["GmSc", "size"]
# data_group["FGA", "sum"]

data_group["FTA"] = np.where(data_group["FTA"] == 0, 1, data_group["FTA"])
data_group["FT%"] = data_group["FT", "sum"]**2 / data_group["FTA", "sum"] 
# data_group["FTA", "sum"]

data_group["TOV_G"] = data_group["TOV", "sum"] / data_group["GmSc", "size"]

In [8]:
column_names_to_normalize = ['PTS_G', 'TRB_G', 'BLK_G', 'STL_G', 'AST_G', '3P_G', 'FG%', 'FT%', 'TOV_G']
x = data_group[column_names_to_normalize].values
scaler = MinMaxScaler()
x_scaled = scaler.fit_transform(x)
df_temp = pd.DataFrame(x_scaled, columns=column_names_to_normalize, index = data_group.index)
data_group[column_names_to_normalize] = df_temp

In [9]:
data_group["sum"] = data_group["PTS_G"] + data_group["TRB_G"] + data_group["BLK_G"] + data_group["STL_G"] + data_group["AST_G"] + \
                    data_group["3P_G"] + data_group["FG%"] + data_group["FT%"] - data_group["TOV_G"] 
data_group = data_group.sort_values(by=['sum'], ascending=False)

In [10]:
data_group1 = data_group.loc[:, ['PTS_G', 'TRB_G', 'BLK_G', 'STL_G', 'AST_G', '3P_G', 'FG%', 'FT%', 'TOV_G', 'sum']]

In [11]:
team = pd.read_csv(r'C:\Users\stang\Google Drive\Data_for_fun\Fantasy Basketball\FBteam.csv', encoding='cp1252')

In [12]:
team_stats = data_group1[data_group1.index.isin(team["Player"])]
total = team_stats.sum(axis=0)

In [13]:
team_stats

,PTS_G,TRB_G,BLK_G,STL_G,AST_G,3P_G,FG%,FT%,TOV_G,sum
,,,,,,,,,,
Player,,,,,,,,,,
Paul George,0.785028,0.517766,0.178451,1.000000,0.371619,0.768953,0.767882,0.572317,0.496575,4.465441
Kemba Walker,0.687243,0.284309,0.172078,0.547714,0.493513,0.650980,0.672407,0.445243,0.465631,3.487856
LeBron James,0.734606,0.562049,0.234082,0.591158,0.669377,0.410412,0.586823,0.281239,0.632660,3.437086
Nikola Vucevic,0.560179,0.782327,0.416145,0.401684,0.336619,0.222307,0.679777,0.195776,0.366619,3.228196
Marc Gasol,0.421345,0.553723,0.452991,0.488000,0.402587,0.259928,0.271044,0.249250,0.400685,2.698183
Victor Oladipo,0.512140,0.363156,0.113248,0.720000,0.454744,0.400722,0.274564,0.160803,0.421233,2.578144
Eric Bledsoe,0.442391,0.294711,0.138986,0.648000,0.480939,0.313306,0.344761,0.198040,0.386375,2.474759
Montrezl Harrell,0.434248,0.432203,0.521394,0.402712,0.158129,0.003304,0.386082,0.238149,0.263292,2.312928


In [14]:
total.sort_values()

FT%         3.207553
BLK_G       4.199919
3P_G        4.978971
AST_G       4.994326
TOV_G       5.138238
FG%         5.274269
TRB_G       6.285342
PTS_G       7.116408
STL_G       8.024543
sum        38.943092
dtype: float64

### Last 10 Games

In [15]:
data_group_10 = gamelogs[gamelogs['Game_Num'] <= 10]
data_group_10 = data_group_10.groupby(by=["Player"]).agg({'GmSc':'size','PTS':['sum'],'TRB':['sum'],'BLK':['sum'], \
                                                    'STL':['sum'], 'AST':['sum'], '3P':['sum'], 'FG':['sum'], 'FGA':['sum'], \
                                                    'FT':['sum'], 'FTA':['sum'], 'TOV':['sum']})

In [16]:
data_group_10["PTS_G"] = data_group_10["PTS", "sum"] / data_group_10["GmSc", "size"]
data_group_10["TRB_G"] = data_group_10["TRB", "sum"] / data_group_10["GmSc", "size"]
data_group_10["BLK_G"] = data_group_10["BLK", "sum"] / data_group_10["GmSc", "size"]
data_group_10["STL_G"] = data_group_10["STL", "sum"] / data_group_10["GmSc", "size"]
data_group_10["AST_G"] = data_group_10["AST", "sum"] / data_group_10["GmSc", "size"]
data_group_10["3P_G"] = data_group_10["3P", "sum"] / data_group_10["GmSc", "size"]

data_group_10["FGA"] = np.where(data_group_10["FGA"] == 0, 1, data_group_10["FGA"])
data_group_10["FG%"] = data_group_10["FG", "sum"]**2 / data_group_10["GmSc", "size"]
# data_group["FGA", "sum"]

data_group_10["FTA"] = np.where(data_group_10["FTA"] == 0, 1, data_group_10["FTA"])
data_group_10["FT%"] = data_group_10["FT", "sum"]**2 / data_group_10["FTA", "sum"] 
# data_group["FTA", "sum"]

data_group_10["TOV_G"] = data_group_10["TOV", "sum"] / data_group_10["GmSc", "size"]

In [17]:
x_10 = data_group_10[column_names_to_normalize].values
x_scaled_10 = scaler.fit_transform(x_10)
df_temp_10 = pd.DataFrame(x_scaled_10, columns=column_names_to_normalize, index = data_group_10.index)
data_group_10[column_names_to_normalize] = df_temp_10

In [18]:
data_group_10["sum"] = data_group_10["PTS_G"] + data_group_10["TRB_G"] + data_group_10["BLK_G"] + data_group_10["STL_G"] + \
                        data_group_10["AST_G"] + data_group_10["3P_G"] + data_group_10["FG%"] + data_group_10["FT%"] - \
                        data_group_10["TOV_G"] 
data_group_10 = data_group_10.sort_values(by=['sum'], ascending=False)

In [19]:
data_group_10 = data_group_10.loc[:, ['PTS_G', 'TRB_G', 'BLK_G', 'STL_G', 'AST_G', '3P_G', 'FG%', 'FT%', 'TOV_G', 'sum']]

In [20]:
team_stats_10 = data_group_10[data_group_10.index.isin(team["Player"])]
total_10 = team_stats_10.sum(axis=0)

In [21]:
team_stats_10

,PTS_G,TRB_G,BLK_G,STL_G,AST_G,3P_G,FG%,FT%,TOV_G,sum
,,,,,,,,,,
Player,,,,,,,,,,
Paul George,0.952632,0.567568,0.107143,0.875000,0.362963,1.000000,0.917050,0.743065,0.54,4.985420
Kemba Walker,0.652632,0.337838,0.178571,0.500000,0.400000,0.583333,0.482907,0.526243,0.44,3.221524
LeBron James,0.594737,0.756757,0.142857,0.541667,0.681481,0.283333,0.494757,0.331574,0.72,3.107164
Nikola Vucevic,0.528947,0.851351,0.392857,0.333333,0.311111,0.133333,0.568874,0.147944,0.42,2.847752
Al Horford,0.389474,0.567568,0.607143,0.583333,0.400000,0.183333,0.303433,0.082849,0.32,2.797132
Montrezl Harrell,0.465789,0.425676,0.571429,0.458333,0.162963,0.000000,0.448219,0.118684,0.32,2.331093
Eric Bledsoe,0.421053,0.331081,0.107143,0.708333,0.355556,0.333333,0.258546,0.182085,0.40,2.297130
Trevor Ariza,0.426316,0.310811,0.071429,0.583333,0.303704,0.416667,0.241597,0.179982,0.28,2.253838


In [22]:
total_10.sort_values()

FT%         3.490377
BLK_G       3.857143
AST_G       4.518519
3P_G        4.633333
FG%         5.102269
TOV_G       5.540000
TRB_G       6.621622
PTS_G       6.957895
STL_G       7.541667
sum        37.182825
dtype: float64

### Available Players

In [23]:
rostered = pd.read_csv(r'C:\Users\stang\Google Drive\Data_for_fun\Fantasy Basketball\Rostered_Players.csv', encoding='cp1252')

In [24]:
IR = ['John Wall', 'Caris LeVert']

In [25]:
available = data_group1[~data_group1.index.isin(rostered["Player"])]
available = available[~available.index.isin(IR)]

In [26]:
available.sort_values(by='sum', ascending=False).head(25)
# Robert Covington
# Larry Nance
# Kent Bazemore (out two weeks)
# JaVale McGee (injured)
# Otto Porter
# Brook Lopez
# Dennis Schroder
# Nemanja Bjelica

,PTS_G,TRB_G,BLK_G,STL_G,AST_G,3P_G,FG%,FT%,TOV_G,sum
,,,,,,,,,,
Player,,,,,,,,,,
Robert Covington,0.360050,0.358906,0.426785,0.955636,0.114686,0.472596,0.102977,0.106030,0.252179,2.645488
Brook Lopez,0.332008,0.278969,0.773210,0.238345,0.095602,0.473920,0.175938,0.098485,0.194497,2.271981
Kent Bazemore,0.359530,0.248347,0.310294,0.693209,0.219014,0.272017,0.157331,0.146388,0.361262,2.044869
Kelly Oubre,0.381456,0.290128,0.306728,0.439448,0.098637,0.289058,0.226404,0.250449,0.248701,2.033607
Terrence Ross,0.402648,0.207553,0.140584,0.387310,0.138092,0.480642,0.276209,0.142885,0.178555,1.997369
Marvin Williams,0.284563,0.368320,0.269549,0.400582,0.105618,0.393436,0.124201,0.074607,0.094147,1.926729
Larry Nance,0.244155,0.528333,0.249608,0.581878,0.264045,0.083548,0.096252,0.111657,0.252866,1.906609
Justin Holiday,0.280808,0.265696,0.188566,0.613895,0.176030,0.393312,0.115256,0.123692,0.256309,1.900947


In [27]:
available_10 = data_group_10[~data_group_10.index.isin(rostered["Player"])]
available_10 = available_10[~available_10.index.isin(IR)]

In [28]:
available_10.sort_values(by='sum', ascending=False).head(25)

,PTS_G,TRB_G,BLK_G,STL_G,AST_G,3P_G,FG%,FT%,TOV_G,sum
,,,,,,,,,,
Player,,,,,,,,,,
Jonathan Isaac,0.352632,0.493243,0.857143,0.541667,0.066667,0.233333,0.172436,0.249061,0.22,2.746181
Kelly Oubre,0.452632,0.364865,0.357143,0.625000,0.185185,0.200000,0.267236,0.371533,0.32,2.503594
Robert Covington,0.371053,0.364865,0.357143,0.791667,0.140741,0.416667,0.165470,0.157807,0.28,2.485411
Brook Lopez,0.339474,0.358108,0.714286,0.458333,0.103704,0.333333,0.172436,0.110161,0.24,2.349835
Mitchell Robinson,0.255263,0.506757,1.000000,0.375000,0.029630,0.000000,0.120727,0.091827,0.08,2.299204
Terrence Ross,0.481579,0.270270,0.178571,0.250000,0.088889,0.550000,0.312841,0.201827,0.12,2.213978
Patrick Beverley,0.260526,0.432432,0.285714,0.541667,0.318519,0.283333,0.069018,0.225439,0.22,2.196648
Terry Rozier,0.323684,0.344595,0.107143,0.541667,0.362963,0.366667,0.132792,0.166437,0.20,2.145947
